In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os
import random
import time
import subprocess

# Function to test for common web vulnerabilities in mobile apps
def test_mobile_web_vulnerabilities(url):
    print(f"Testing mobile web vulnerabilities for {url}...")

    # SQL Injection
    print("Testing SQL Injection...")
    payload = "' OR 1=1 --"
    response = requests.get(f"{url}/login?username=admin&password={payload}")
    if "Login successful" in response.text:
        print("SQL Injection vulnerability detected.")
    
    # Cross-Site Scripting (XSS)
    print("Testing for XSS...")
    payload = "<script>alert('XSS')</script>"
    response = requests.get(f"{url}/search?q={payload}")
    if payload in response.text:
        print("XSS vulnerability detected.")
    
    # Insecure Communication (checking if HTTP is used)
    print("Testing for insecure communication (HTTP)...")
    if url.startswith("http://"):
        print("Insecure HTTP connection detected. Switch to HTTPS.")
    
    # Cross-Site Request Forgery (CSRF)
    print("Testing for CSRF...")
    csrf_token = get_csrf_token(url)
    if not csrf_token:
        print("CSRF vulnerability detected.")
    
    # Local File Inclusion (LFI)
    print("Testing for LFI...")
    payload = "../../../../etc/passwd"
    response = requests.get(f"{url}/file.php?file={payload}")
    if "root:" in response.text:
        print("LFI vulnerability detected.")

# Function to extract CSRF token from a web page (simplified)
def get_csrf_token(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    token = soup.find("input", {"name": "csrf_token"})
    return token['value'] if token else None

# Function to analyze mobile app security via APK
def analyze_apk(apk_path):
    print(f"Analyzing APK: {apk_path}")
    if not os.path.exists(apk_path):
        print("APK file not found.")
        return

    # Extract app data using APKTool
    apktool_cmd = f"apktool d {apk_path} -o extracted_apk"
    subprocess.run(apktool_cmd, shell=True)

    # Check for sensitive information in the source code
    print("Scanning for sensitive data...")
    for root, dirs, files in os.walk('extracted_apk'):
        for file in files:
            if file.endswith(".smali"):
                with open(os.path.join(root, file), 'r') as f:
                    content = f.read()
                    if 'password' in content or 'secret' in content:
                        print(f"Sensitive data found in {file}")

    # Check for insecure permissions in AndroidManifest.xml
    print("Checking permissions...")
    manifest_path = 'extracted_apk/AndroidManifest.xml'
    if os.path.exists(manifest_path):
        with open(manifest_path, 'r') as f:
            content = f.read()
            if 'INTERNET' in content and 'ACCESS_FINE_LOCATION' in content:
                print("Potential security risk: App has internet and location access permissions.")

# Example usage of mobile pentesting tools
url = "http://example.com"
apk_path = "example.apk"

# Test for mobile web vulnerabilities
test_mobile_web_vulnerabilities(url)

# Analyze mobile app APK for security flaws
analyze_apk(apk_path)